In [2]:
import pandas as pd 

In [3]:
sov_path = "data/raw/c0{county}_g{year}_sov_data_by_g{year}_srprec.csv"
sor_path = "data/raw/c0{county}_g{year}_voters_by_g{year}_srprec.csv"
reg_path = "data/raw/c0{county}_g{year}_registration_by_g{year}_srprec.csv"

sov_san_luis = []
sov_santa_barbara = []
sor_san_luis = []
sor_santa_barbara = []
reg_san_luis = []
reg_santa_barbara = []

df_sov_sb_combined = None
df_sov_sl_combined = None

df_election_totals = pd.DataFrame()
df_registration_voted_totals = pd.DataFrame()
df_registration_totals = pd.DataFrame()

# Iterate over election cycles.
for year in ["12", "14", "16", "18", "20"]:
    
    # Read sov data for each county.
    df_sov_sb = pd.read_csv(sov_path.format(county="83", year=year))
    df_sov_sl = pd.read_csv(sov_path.format(county="79", year=year))

    # Read sor data for each county.
    df_sor_sb = pd.read_csv(sor_path.format(county="83", year=year))
    df_sor_sl = pd.read_csv(sor_path.format(county="79", year=year))

    # Read registration information for each county.
    df_reg_sb = pd.read_csv(reg_path.format(county="83", year=year))
    df_reg_sl = pd.read_csv(reg_path.format(county="79", year=year))

    # Filter precincts to only those that are in the correct district.
    df_sov_sb = df_sov_sb[df_sov_sb["addist"] == 35]
    df_sov_sl = df_sov_sl[df_sov_sl["addist"] == 35]

    # SOR files don't contain information about what AD they belong to. 
    # So we need to filter them by the related SOV file. 
    df_sor_sb = df_sor_sb[df_sor_sb["srprec"].isin(df_sov_sb["srprec"])]
    df_sor_sl = df_sor_sl[df_sor_sl["srprec"].isin(df_sor_sl["srprec"])]

    # Likewise, registration files do not include assembly district to filter on.
    df_reg_sb = df_reg_sb[df_reg_sb["srprec"].isin(df_sov_sb["srprec"])]
    df_reg_sl = df_reg_sl[df_reg_sl["srprec"].isin(df_sov_sl["srprec"])]

    print(df_reg_sl.shape[0], df_sor_sl.shape[0])

    # Get totals. We can't use the reported county totals - because they don't
    # necessarily reflect the totals of assembly districts.
    sr_sov_sb_totals = df_sov_sb.iloc[:, 6:].sum()
    sr_sov_sl_totals = df_sov_sl.iloc[:, 6:].sum()
    sr_cycle_totals = sr_sov_sb_totals + sr_sov_sl_totals

    # Add the cycle year to summated Series. 
    sr_year = pd.Series(data=["20" + year], index=["year"])
    sr_cycle_totals = sr_cycle_totals.append(sr_year)

    # Total information about registration for those who voted.
    sr_sor_sb_totals = df_sor_sb.iloc[:, 3:].sum()
    sr_sor_sl_totals = df_sor_sl.iloc[:, 3:].sum()
    sr_reg_vote_totals = sr_sor_sb_totals + sr_sor_sl_totals

    sr_reg_vote_totals = sr_reg_vote_totals.append(sr_year)

    # Total registration information for all voters in the district.
    sr_reg_sb_totals = df_reg_sb.iloc[:, 3:].sum()
    sr_reg_sl_totals = df_reg_sl.iloc[:, 3:].sum()
    sr_reg_totals = sr_reg_sb_totals + sr_reg_sl_totals

    sr_reg_totals = sr_reg_totals.append(sr_year)

    # Add summated totals to election total
    df_election_totals = df_election_totals.append(sr_cycle_totals, ignore_index=True)
    df_registration_voted_totals = df_registration_voted_totals.append(sr_reg_vote_totals, ignore_index=True)
    df_registration_totals = df_registration_totals.append(sr_reg_totals, ignore_index=True)
   
    # Determine the previous cycle - used to suffix columns labels during merge
    prev = str(int(year) - 2)

    # Merge datasets by precinct.  
    df_sov_sb_combined = df_sov_sb if df_sov_sb_combined is None else \
        df_sov_sb_combined.merge(
            right=df_sov_sb, 
            on="srprec", 
            how="inner", 
            suffixes=["_{}".format(prev), "_{}".format(year)])

    df_sov_sl_combined = df_sov_sl if df_sov_sl_combined is None else \
        df_sov_sl_combined.merge(
            right=df_sov_sl, 
            on="srprec", 
            how="inner", 
            suffixes=["_{}".format(prev), "_{}".format(year)])

    df_sov_sb["year"] = "20{}".format(year)
    df_sov_sl["year"] = "20{}".format(year)

    sov_san_luis.append(df_sov_sl)
    sov_santa_barbara.append(df_sov_sb)

df_election_totals.set_index("year", inplace=True)
df_registration_voted_totals.set_index("year", inplace=True)
df_registration_totals.set_index("year", inplace=True)

# df_election_totals
#df_registration_voted_totals
df_registration_totals - df_registration_voted_totals


142 142
161 161
153 153
152 152
103 103


,aip,chidcl,chidem,chioth,chirep,dcl,dclf1824,dclf2534,dclf3544,dclf4554,...,rreg5g,rreg6g,rreg7g,rreg8g,rreg9g,totreg_r,vietdcl,vietdem,vietoth,vietrep
year,,,,,,,,,,,,,,,,,,,,,
2012,1821.0,109.0,72.0,25.0,79.0,14647.0,2070.0,1953.0,1042.0,910.0,...,2112.0,693.0,808.0,463.0,2435.0,48464.0,31.0,29.0,9.0,36.0
2014,3427.0,182.0,125.0,30.0,105.0,26406.0,3007.0,3536.0,2258.0,2016.0,...,1560.0,3256.0,1020.0,1436.0,4927.0,90187.0,69.0,56.0,16.0,45.0
2016,1888.0,128.0,90.0,12.0,77.0,15311.0,1772.0,2045.0,1135.0,939.0,...,1233.0,430.0,1170.0,373.0,2552.0,48249.0,54.0,56.0,9.0,28.0
2018,2826.0,230.0,125.0,18.0,81.0,27321.0,2965.0,3537.0,2343.0,1704.0,...,760.0,1156.0,422.0,1165.0,3142.0,77639.0,93.0,74.0,10.0,41.0
2020,1985.0,104.0,72.0,14.0,53.0,15143.0,1497.0,2031.0,1245.0,723.0,...,375.0,228.0,278.0,105.0,1249.0,43442.0,44.0,37.0,16.0,23.0


In [10]:
df_concat_sb = pd.concat(sov_santa_barbara, axis=0)
df_filtered_sb = df_concat_sb.groupby("srprec").filter(lambda x : len(x) > 2)
# print(df_filtered_sb["year"].unique())
print("Filtered precincts {} vs {} total precincts".format(df_filtered_sb.shape[0], df_concat_sb.shape[0]))

df_concat_slo = pd.concat(sov_san_luis, axis=0)
df_filtered_slo = df_concat_slo.groupby("srprec").filter(lambda x : len(x) > 4)
print("Filtered precincts {} vs {} total precincts".format(df_filtered_slo.shape[0], df_concat_slo.shape[0]))


df_combined_precincts = pd.concat([df_concat_sb, df_concat_slo])
df_precincts_over_time = df_combined_precincts.groupby("srprec").filter(lambda x : len(x) > 4)
print("Filtered precincts {} vs {} total precincts".format(df_filtered.shape[0], df_combined.shape[0]))


Filtered precincts 171 vs 431 total precincts
Filtered precincts 380 vs 725 total precincts
Filtered precincts 380 vs 1156 total precincts


## Validating Results   

We can perform a basic sanity check on the results of our filtered data, by comparing our own summated results of the assembly races with the certified totals. These totals were retrieved from BallotPedia 

In [9]:

confirmed_totals = { '2020' : [103206, 126579], 
#                     '2018' : [76994, 97749],
                     '2016' : [87168, 105247], 
                     '2014' : [46126, 77452],
                     '2012' : [65500, 103762]}


for yr in confirmed_totals:
    demVotes = df_election_totals.loc[yr]["ASSDEM01"]
    repVotes = df_election_totals.loc[yr]["ASSREP01"]
    demExpected = confirmed_totals[yr][0]
    repExpected = confirmed_totals[yr][1]
    assert demExpected == demVotes, "({}) Data was {} but expected: {}".format(yr, demVotes, demExpected)
    assert repExpected == repVotes, "({}) Data was {} but expected: {}".format(yr, repVotes, repExpected)

Interestingly, and rather frustratingly, it appears that Santa Barbara's 2018 county file does not actually contain information about the assembly's votes.

## Exporting Results

In [12]:
df_election_totals.to_csv("data/processed/election_totals.csv")
df_registration_voted_totals.to_csv("data/processed/registration_voted_totals.csv")
df_registration_totals.to_csv("data/processed/registration_totals.csv")
df_precincts_over_time.to_csv("data/processed/precincts_over_time.csv")